In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ==========================================================
# MULTI-LANGUAGE TRANSLATION (ENG + FRA → All Targets)
# Output: ONLY 2 CSV files inside output_judge
# ==========================================================

# === IMPORT STATEMENTS ===
# What this does: Loads all necessary libraries for machine translation
# Why we need it: Each library provides specific functionality we need to translate text

# torch: PyTorch library - provides tensor operations and GPU support
# Why: We need this to run the neural network model on GPU/CPU and handle numerical computations
import torch

# transformers: Hugging Face library for pre-trained NLP models
# AutoTokenizer: Converts text into numbers (tokens) that the model understands
# AutoModelForSeq2SeqLM: Loads a sequence-to-sequence language model (translates one language to another)
# Why: NLLB model is a translation model that needs tokenization and model loading
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# pandas: Data manipulation library - works with tables/dataframes like Excel
# Why: Our dataset is a CSV file, and we need to filter, organize, and save translation results
import pandas as pd

# time: Built-in Python library for measuring execution time
# Why: We want to track how long translations take (performance monitoring)
import time

# datetime: Built-in Python library for date/time operations
# Why: We print timestamps to know when the script ran (for logging/debugging)
from datetime import datetime

# tqdm: Progress bar library - shows visual progress during long operations
# Why: Translation takes time, so we show a progress bar so users know it's working
from tqdm import tqdm

# pathlib.Path: Modern way to handle file/folder paths (works on Windows/Mac/Linux)
# Why: We need to create output folders and save files - Path makes this easier and safer
from pathlib import Path

# === LANGUAGE CONFIGURATION ===
# What this does: Defines which languages we'll translate to and their codes
# Why we need it: The translation model needs specific language codes, and we need to organize our target languages
# Input: None (configuration data)
# Output: Language codes and mappings used throughout the script

# TARGET_LANGUAGES: List of ISO 639-3 language codes we want to translate INTO
# Why: We translate FROM English/French TO these 14 languages
# Format: 3-letter ISO codes (e.g., "jpn" = Japanese, "zho" = Mandarin Chinese)
TARGET_LANGUAGES = [
    "jpn",  # Japanese
    "zho",  # Mandarin
    "kor",  # Korean
    "hin",  # Hindi
    "dan",  # Danish
    "nld",  # Dutch
    "fin",  # Finnish
    "deu",  # German
    "ell",  # Greek
    "ita",  # Italian
    "pol",  # Polish
    "por",  # Portuguese
    "spa",  # Spanish
    "swe",  # Swedish
]

# PRIORITY_LANGUAGES: Source languages we translate FROM
# Why: These are our starting points - we have English and French source texts
# Business reason: We want to test translation quality from multiple source languages
PRIORITY_LANGUAGES = ["eng", "fra"]

# DEFAULT_SCRIPTS: Maps language codes to their writing system codes (ISO 15924)
# Why: Some languages have multiple writing systems (e.g., Chinese has Simplified/Hanzi)
# Technical reason: NLLB model needs both language code AND script code to translate correctly
# Example: "zho" (Chinese) + "Hans" (Simplified) = "zho_Hans" for the model
DEFAULT_SCRIPTS = {
    "eng": "Latn",  # Latin script (A-Z alphabet)
    "fra": "Latn",  # Latin script
    "jpn": "Jpan",  # Japanese script (Hiragana, Katakana, Kanji)
    "kor": "Hang",  # Hangul script (Korean alphabet)
    "hin": "Deva",  # Devanagari script (Hindi)
    "dan": "Latn",  # Latin script
    "nld": "Latn",  # Latin script
    "fin": "Latn",  # Latin script
    "deu": "Latn",  # Latin script
    "ell": "Grek",  # Greek script
    "ita": "Latn",  # Latin script
    "pol": "Latn",  # Latin script
    "por": "Latn",  # Latin script
    "spa": "Latn",  # Latin script
    "swe": "Latn",  # Latin script
    "zho": "Hans",  # Simplified Chinese (Han script)
}

# LANGUAGE_NAMES: Human-readable names for each language code
# Why: Makes output messages readable (shows "Japanese" instead of "jpn")
# Business reason: Better user experience when printing progress messages
LANGUAGE_NAMES = {
    "eng": "English",
    "fra": "French",
    "jpn": "Japanese",
    "kor": "Korean",
    "hin": "Hindi",
    "dan": "Danish",
    "nld": "Dutch",
    "fin": "Finnish",
    "deu": "German",
    "ell": "Greek",
    "ita": "Italian",
    "pol": "Polish",
    "por": "Portuguese",
    "spa": "Spanish",
    "swe": "Swedish",
    "zho": "Mandarin"
}

# Function: get_nllb_code
# What this does: Combines language code and script code into NLLB model format
# Why: NLLB model requires format like "eng_Latn" not just "eng"
# Input: iso_639_3 (3-letter language code), script (4-letter script code)
# Output: Combined string like "eng_Latn" or "zho_Hans"
def get_nllb_code(iso_639_3, script):
    return f"{iso_639_3}_{script}"


# === MODEL SETUP ===
# What this does: Loads the NLLB translation model and prepares it for use
# Why we need it: We need a pre-trained model that can translate between 200+ languages
# Input: Model name from Hugging Face
# Output: Loaded tokenizer and model ready for translation

# Print header with timestamp for logging
# Why: Helps track when script ran and what it's doing (useful for debugging)
print("=" * 80)
print("🚀 NLLB-200-600M TRANSLATION")
print("=" * 80)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Device selection: Check if GPU (CUDA) is available, otherwise use CPU
# Why: GPUs are 10-100x faster for neural networks, but CPU works if no GPU
# Technical reason: Neural networks do lots of parallel math - GPUs excel at this
# Business reason: Faster translation = less waiting time
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

print("\n⏳ Loading NLLB model...")

# Model name: NLLB-200-Distilled-600M from Facebook/Meta
# What: Pre-trained translation model supporting 200+ languages
# Why this model: It's specifically designed for many languages, and "distilled" means smaller/faster
# "600M" = 600 million parameters (model size - bigger = better quality but slower)
model_name = "facebook/nllb-200-distilled-600M"

# Tokenizer: Converts text into tokens (numbers) that the model understands
# Why: Models work with numbers, not text directly
# Input: Text strings like "Hello world"
# Output: Token IDs like [1234, 5678] that represent the text
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Model: The actual neural network that does translation
# from_pretrained: Downloads and loads the pre-trained weights from Hugging Face
# torch_dtype: Data type for model weights
#   - float16 on GPU: Uses less memory, faster (half precision)
#   - float32 on CPU: More accurate, required for CPU compatibility
# .to(device): Moves model to GPU or CPU
# Why: Model weights are large - float16 saves memory, GPU speeds up computation
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device)

print("✓ Model loaded successfully!")


# === LOAD DATASET ===
# What this does: Reads the CSV file and filters sentences by language
# Why we need it: We need source texts in English and French to translate
# Input: CSV file at "data/df.csv" with columns: id, iso_639_3, iso_15924, text
# Output: Dictionary of dataframes, one per language, with aligned sentence IDs

print("\n⏳ Loading dataset...")

# Read CSV file into pandas DataFrame
# Input: CSV file with multilingual sentences
# Output: DataFrame with columns like id, iso_639_3 (language code), iso_15924 (script), text
# Why: CSV is a common data format - easy to share and edit
df = pd.read_csv("data/df.csv")

# Filter language data: Separate sentences by language
# Why: We need to organize sentences by language so we can translate from English/French to others
# Input: Full dataframe with all languages mixed together
# Output: Dictionary where each key is a language code and value is a filtered dataframe
lang_dfs = {}

# Loop through all languages we care about (source + target languages)
# Why: We need to filter the dataset to only include languages we'll use
for lang_code in PRIORITY_LANGUAGES + TARGET_LANGUAGES:
    # Get the script code for this language (e.g., "Latn" for English)
    script = DEFAULT_SCRIPTS[lang_code]
    
    # Filter dataframe: Keep only rows matching this language AND script
    # Why: Some languages have multiple scripts (e.g., Chinese Simplified vs Traditional)
    # Technical reason: We need exact matches to ensure correct language/script combination
    # Input: Full dataframe
    # Output: Filtered dataframe with only this language's sentences
    filtered = df[
        (df["iso_639_3"] == lang_code) &  # Match language code
        (df["iso_15924"] == script)        # Match script code
    ].copy()  # .copy() prevents warnings about modifying views
    
    # Only store if we found sentences for this language
    if len(filtered) > 0:
        # Sort by ID so sentences are in order (important for alignment)
        # Why: We need aligned sentences - same ID = same sentence in different languages
        lang_dfs[lang_code] = filtered.sort_values("id")
        print(f"✓ Loaded {len(filtered)} sentences for {LANGUAGE_NAMES[lang_code]}")

# Ensure aligned IDs: Find sentences that exist in BOTH English and French
# Why: We need the same sentences in both source languages for fair comparison
# Business reason: We want to compare "How well does model translate English vs French?"
# Technical reason: We can only compare if we have the same source sentences
# Input: Sets of IDs from English and French dataframes
# Output: List of common IDs (sentences present in both languages)
common_ids = set(lang_dfs["eng"]["id"]).intersection(set(lang_dfs["fra"]["id"]))
# Sort IDs and limit to first 1000 (for faster processing/testing)
# Why: Limits dataset size - can remove [:1000] to use all sentences
common_ids = sorted(list(common_ids))[:1000]

# Filter all language dataframes to only include aligned sentences
# Why: We only want sentences that exist in both English and French sources
# Input: Full language dataframes
# Output: Filtered dataframes with only common IDs, sorted by ID
# Data transformation: Each dataframe → filtered to common_ids → sorted
for lang_code in lang_dfs:
    lang_dfs[lang_code] = lang_dfs[lang_code][
        lang_dfs[lang_code]["id"].isin(common_ids)  # Keep only IDs in common_ids
    ].sort_values("id")  # Sort by ID for consistency

print(f"\n✓ Using {len(common_ids)} aligned sentences")


# === TRANSLATION FUNCTIONS ===
# What this does: Contains functions to translate text batches and measure performance
# Why we need it: Core translation logic - converts source text to target language
# Input: List of source texts, target language code
# Output: List of translated texts

# Function: batch_translate
# What this does: Translates multiple texts at once (batching) for efficiency
# Why: Processing multiple texts together is faster than one-by-one (GPU parallelization)
# Business reason: Faster processing = lower compute costs and faster results
# Input: 
#   - texts: List of source text strings (e.g., ["Hello", "How are you?"])
#   - tgt_lang: Target language code in NLLB format (e.g., "jpn_Jpan" for Japanese)
#   - batch_size: How many texts to process at once (default 8)
# Output: List of translated text strings (same length as input)
def batch_translate(texts, tgt_lang, batch_size=8):
    # forced_bos_id: "Beginning of Sequence" token ID for target language
    # What: Special token that tells model "start generating in this language"
    # Why: NLLB supports 200+ languages - this token forces correct target language
    # Technical reason: Without this, model might generate wrong language
    forced_bos_id = tokenizer.convert_tokens_to_ids(tgt_lang)
    
    # hypotheses: Will store all translated texts
    hypotheses = []

    # Process texts in batches (e.g., 8 at a time)
    # Why: GPUs work best with batches - processes multiple items in parallel
    # tqdm: Shows progress bar so user knows how much is done
    # Input: Range from 0 to len(texts), stepping by batch_size
    for i in tqdm(range(0, len(texts), batch_size)):
        # Extract batch: Get next batch_size texts (e.g., texts[0:8], then texts[8:16])
        # Data transformation: Full list → slice of batch_size items
        batch = texts[i:i + batch_size]

        # Tokenize: Convert text strings to token IDs (numbers)
        # return_tensors="pt": Return PyTorch tensors (not NumPy arrays)
        # padding=True: Add padding tokens so all texts in batch have same length
        #   Why: Neural networks need fixed-size inputs - padding makes short texts match long ones
        # .to(device): Move tensors to GPU or CPU (wherever model is)
        # Input: List of text strings
        # Output: Tensor of token IDs, shape [batch_size, max_length]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True
        ).to(device)

        # torch.no_grad(): Disable gradient calculation (we're not training)
        # Why: Saves memory and speeds up inference - gradients only needed for training
        with torch.no_grad():
            # model.generate(): Run the model to generate translations
            # **inputs: Unpack tokenized inputs (passes them to model)
            # forced_bos_token_id: Force model to start with target language token
            # max_length=128: Maximum tokens in output (prevents very long translations)
            #   Why: Limits output size - prevents model from generating extremely long text
            # num_beams=4: Beam search width (explores 4 best translation paths)
            #   What: Beam search tries multiple translation options and picks best
            #   Why: Better quality than greedy decoding (picking first option)
            #   Trade-off: Higher num_beams = better quality but slower
            # Input: Tokenized source texts
            # Output: Token IDs of translated texts, shape [batch_size, generated_length]
            outputs = model.generate(
                **inputs,
                forced_bos_token_id=forced_bos_id,
                max_length=128,
                num_beams=4
            )

        # Decode: Convert token IDs back to text strings
        # batch_decode: Decode entire batch at once
        # skip_special_tokens=True: Remove special tokens like [PAD], [BOS] from output
        #   Why: These are model-internal tokens, not part of actual translation
        # Input: Tensor of token IDs
        # Output: List of translated text strings
        # Data transformation: Token IDs [batch_size, length] → text strings [batch_size]
        hypotheses.extend(
            tokenizer.batch_decode(outputs, skip_special_tokens=True)
        )

    return hypotheses


# Function: time_translate
# What this does: Wrapper that translates texts and measures how long it takes
# Why: Performance monitoring - helps understand translation speed
# Business reason: Need to know processing time for planning and cost estimation
# Input:
#   - texts: List of source text strings
#   - tgt_lang: Target language code
# Output: Tuple of (translated_texts, elapsed_time_in_seconds)
def time_translate(texts, tgt_lang):
    # Clear GPU cache if using CUDA (GPU)
    # Why: Frees up GPU memory from previous operations
    # Technical reason: Prevents "out of memory" errors during long runs
    if device == "cuda":
        torch.cuda.empty_cache()

    # Record start time
    start = time.time()
    
    # Perform translation
    hyps = batch_translate(texts, tgt_lang)
    
    # Calculate elapsed time
    elapsed = time.time() - start

    # Return both translations and timing info
    return hyps, elapsed


# === CREATE OUTPUT FOLDER ===
# What this does: Creates a folder to save translation results
# Why we need it: We need a place to save the CSV files with translations
# Input: Folder path "results"
# Output: Creates folder if it doesn't exist (no error if it already exists)

# Define output directory path
# Why: Centralized location for all output files - easier to find results
output_dir = Path("results")

# Create folder (and parent folders if needed)
# parents=True: Create parent directories if they don't exist
# exist_ok=True: Don't raise error if folder already exists
# Why: Makes script robust - works whether folder exists or not
output_dir.mkdir(parents=True, exist_ok=True)
print("\n📁 Created folder: results")


# === TRANSLATION PHASE ===
# What this does: Performs all translations from English and French to all target languages
# Why we need it: This is the main work - translating source texts to create dataset
# Business reason: We need translations to evaluate model quality (LLM as a Judge)
# Input: Source texts in English and French
# Output: Lists of translation dictionaries (will be saved to CSV)

# Extract source texts: Convert dataframe columns to lists
# Why: Translation function expects lists, not dataframes
# Data transformation: DataFrame column → Python list
# Input: DataFrame with "text" column
# Output: List of text strings
sources_eng = lang_dfs["eng"]["text"].tolist()
sources_fra = lang_dfs["fra"]["text"].tolist()

# Get NLLB language codes for source languages (not used in this version, but prepared)
# Why: These would be used if we needed to specify source language explicitly
eng_nllb = get_nllb_code("eng", DEFAULT_SCRIPTS["eng"])
fra_nllb = get_nllb_code("fra", DEFAULT_SCRIPTS["fra"])

# Initialize lists to store translation results
# Why: We'll collect all translations before saving to CSV
# Structure: Each item is a dictionary with source, target, original, translated
eng_translations = []
fra_translations = []


# ================= ENGLISH → TARGETS =================
# What this does: Translates all English sentences to each target language
# Why: First source language - we want to compare translation quality from English
# Input: English source texts, target language codes
# Output: List of translation dictionaries

print("\n" + "=" * 60)
print("TRANSLATING FROM ENGLISH")
print("=" * 60)

# Loop through each target language
# Why: We need translations to all 14 target languages
for tgt_lang_code in TARGET_LANGUAGES:

    # Get script code for target language (e.g., "Jpan" for Japanese)
    tgt_script = DEFAULT_SCRIPTS[tgt_lang_code]
    
    # Build NLLB language code (e.g., "jpn_Jpan")
    # Why: Model needs this specific format to know target language
    tgt_nllb = get_nllb_code(tgt_lang_code, tgt_script)

    # Print progress message
    print(f"\n➡ English → {LANGUAGE_NAMES[tgt_lang_code]}")

    # Perform translation: Translate all English texts to this target language
    # Input: List of English texts, target language code
    # Output: Tuple of (translated_texts, elapsed_time)
    hyps, elapsed = time_translate(sources_eng, tgt_nllb)
    
    # Print first translation as example (for verification)
    print(hyps[0])

    # Store results: Create dictionary for each translation pair
    # Why: We need structured data with source language, target language, original, and translation
    # zip(): Pairs up original text with its translation (same index)
    # Data transformation: Two lists → list of dictionaries
    # Input: sources_eng (list), hyps (list of translations)
    # Output: List of dictionaries, each with 4 keys
    for original, translated in zip(sources_eng, hyps):
        eng_translations.append({
            "source_language": "eng",           # Which language we translated from
            "target_language": tgt_lang_code,   # Which language we translated to
            "original_text": original,          # Original English text
            "translated_text": translated       # Translated text in target language
        })

print("✅ English translations complete")


# ================= FRENCH → TARGETS =================
# What this does: Translates all French sentences to each target language
# Why: Second source language - we want to compare translation quality from French
# Business reason: Compare "Does model translate better from English or French?"
# Input: French source texts, target language codes
# Output: List of translation dictionaries

print("\n" + "=" * 60)
print("TRANSLATING FROM FRENCH")
print("=" * 60)

# Loop through each target language (same as English loop)
# Why: Same target languages, but starting from French instead
for tgt_lang_code in TARGET_LANGUAGES:

    # Get script code for target language
    tgt_script = DEFAULT_SCRIPTS[tgt_lang_code]
    
    # Build NLLB language code
    tgt_nllb = get_nllb_code(tgt_lang_code, tgt_script)

    # Print progress message
    print(f"\n➡ French → {LANGUAGE_NAMES[tgt_lang_code]}")

    # Perform translation: Translate all French texts to this target language
    # Input: List of French texts, target language code
    # Output: Tuple of (translated_texts, elapsed_time)
    hyps, elapsed = time_translate(sources_fra, tgt_nllb)
    
    # Print first translation as example
    print(hyps[0])

    # Store results: Create dictionary for each translation pair
    # Same structure as English translations, but source_language is "fra"
    for original, translated in zip(sources_fra, hyps):
        fra_translations.append({
            "source_language": "fra",           # Which language we translated from
            "target_language": tgt_lang_code,   # Which language we translated to
            "original_text": original,          # Original French text
            "translated_text": translated       # Translated text in target language
        })

print("✅ French translations complete")


# === SAVE FINAL CSV FILES ===
# What this does: Converts translation lists to DataFrames and saves as CSV files
# Why we need it: CSV format is easy to share, analyze, and use in other tools
# Business reason: These CSV files are the output dataset for LLM evaluation
# Input: Lists of translation dictionaries
# Output: Two CSV files in the "results" folder

# Convert lists of dictionaries to pandas DataFrames
# Why: DataFrames make it easy to save as CSV and manipulate data
# Data transformation: List of dictionaries → DataFrame (table format)
# Input: eng_translations (list of dicts), fra_translations (list of dicts)
# Output: DataFrames with columns: source_language, target_language, original_text, translated_text
eng_df = pd.DataFrame(eng_translations)
fra_df = pd.DataFrame(fra_translations)

# Save DataFrames to CSV files
# output_dir / "filename.csv": Uses Path object for safe path joining (works on all OS)
# index=False: Don't save row numbers as a column (cleaner CSV)
# Why: CSV files are universal format - can open in Excel, Python, R, etc.
# Expected output location: results/english_translations.csv and results/french_translations.csv
eng_df.to_csv(output_dir / "english_translations.csv", index=False)
fra_df.to_csv(output_dir / "french_translations.csv", index=False)

# Print completion message with file locations
# Why: User needs to know where to find the output files
print("\n" + "=" * 60)
print("🎉 DONE!")
print("Files saved:")
print("📄 output_judge/english_translations.csv")
print("📄 output_judge/french_translations.csv")
print("=" * 60)

C:\Users\sp_hp\miniconda3\envs\mt_benchmark\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 NLLB-200-600M TRANSLATION
Date: 2026-02-23 08:37:29
Device: cuda

⏳ Loading NLLB model...


`torch_dtype` is deprecated! Use `dtype` instead!


✓ Model loaded successfully!

⏳ Loading dataset...


C:\Users\sp_hp\AppData\Local\Temp\ipykernel_11460\710875769.py:111: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/df.csv")


...

Os cientistas da Faculdade de Medicina da Universidade de Stanford anunciaram nesta segunda-feira a criação de uma nova ferramenta de diagnóstico, que permitiria diferenciar as células de acordo com o seu tipo.

➡ French → Spanish


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [05:39<00:00,  2.71s/it]


Científicos de la Escuela de Medicina de la Universidad de Stanford anunciaron este lunes la creación de una nueva herramienta de diagnóstico que permitirá diferenciar las células según su tipo.

➡ French → Swedish


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [05:16<00:00,  2.53s/it]


Forskare vid Stanforduniversitetets medicinska skola har på måndagen meddelat att de ska skapa ett nytt diagnostiskt verktyg som kan skilja cellerna beroende på deras typ.
✅ French translations complete

🎉 DONE!
Files saved:
📄 output_judge/english_translations.csv
📄 output_judge/french_translations.csv
